# Reading netCDF data
- requires [numpy](http://numpy.scipy.org) and netCDF/HDF5 C libraries.
- Github site: https://github.com/Unidata/netcdf4-python
- Online docs: http://unidata.github.io/netcdf4-python/
- Based on Konrad Hinsen's old [Scientific.IO.NetCDF](http://dirac.cnrs-orleans.fr/plone/software/scientificpython/) API, with lots of added netcdf version 4 features.
- Developed by Jeff Whitaker at NOAA, with many contributions from users.

## Interactively exploring a netCDF File

Let's explore a netCDF file from the *Atlantic Real-Time Ocean Forecast System*

first, import netcdf4-python and numpy

In [127]:
import netCDF4
import numpy as np

## Create a netCDF4.Dataset object
- **`f`** is a `Dataset` object, representing an open netCDF file.
- printing the object gives you summary information, similar to *`ncdump -h`*.

In [128]:
f = netCDF4.Dataset('/Users/vsood/Downloads/prods_op_mogreps-uk_20140717_03_11_015.nc', 'r')
print(f) 

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    source: Data from Met Office Unified Model
    um_version: 8.5
    Conventions: CF-1.5
    dimensions(sizes): time(36), grid_latitude(548), grid_longitude(421), time_0(3), pressure(3), time_1(3), bnds(2), pressure_0(2), pressure_1(3), dim0(3), grid_longitude_0(421), grid_latitude_0(547), time_3(3)
    variables(dimensions): float32 stratiform_snowfall_rate(time,grid_latitude,grid_longitude), int32 rotated_latitude_longitude(), float64 time(time), float32 grid_latitude(grid_latitude), float32 grid_longitude(grid_longitude), float64 forecast_period(time), float64 forecast_reference_time(), float32 cloud_base_altitude_assuming_only_consider_cloud_area_fraction_greater_than_2p5_oktas(time_0,grid_latitude,grid_longitude), float64 time_0(time_0), float64 forecast_period_0(time_0), float32 cloud_area_fraction_assuming_maximum_random_overlap(time_0,grid_latitude,grid_longitude), float32 wet_bulb_freezing_

## Access a netCDF variable
- variable objects stored by name in **`variables`** dict. A variable typically depends on one or more ***`dimensions`*** making them multi-dimensional
- print the variable yields summary info (including all the attributes).
- no actual data read yet (just have a reference to the variable object with metadata).

In [129]:
print(f.variables.keys()) # get all variable names
#print(f.variables.values()) # get all variable names
air_temperature = f.variables['air_temperature_2']  # temperature variable
print(air_temperature) 

odict_keys(['stratiform_snowfall_rate', 'rotated_latitude_longitude', 'time', 'grid_latitude', 'grid_longitude', 'forecast_period', 'forecast_reference_time', 'cloud_base_altitude_assuming_only_consider_cloud_area_fraction_greater_than_2p5_oktas', 'time_0', 'forecast_period_0', 'cloud_area_fraction_assuming_maximum_random_overlap', 'wet_bulb_freezing_level_altitude', 'wet_bulb_potential_temperature', 'pressure', 'unknown', 'air_pressure_at_sea_level', 'air_temperature', 'height', 'air_temperature_0', 'time_1', 'time_1_bnds', 'forecast_period_1', 'forecast_period_1_bnds', 'air_temperature_1', 'air_temperature_2', 'pressure_0', 'dew_point_temperature', 'fog_area_fraction', 'geopotential_height', 'pressure_1', 'high_type_cloud_area_fraction', 'low_type_cloud_area_fraction', 'medium_type_cloud_area_fraction', 'relative_humidity', 'relative_humidity_0', 'specific_humidity', 'stratiform_rainfall_amount', 'forecast_period_2', 'forecast_period_2_bnds', 'time_2', 'time_2_bnds', 'stratiform_rain

## List the Dimensions

- All variables in a netCDF file have an associated shape, specified by a list of dimensions.
- Let's list all the dimensions in this netCDF file.
- Note that the **`time`** dimension is special (*`unlimited`*), which means it can be appended to.

In [130]:
for d1 in f.dimensions.keys():
    print(d1)

time
grid_latitude
grid_longitude
time_0
pressure
time_1
bnds
pressure_0
pressure_1
dim0
grid_longitude_0
grid_latitude_0
time_3


In [131]:
for d2 in f.dimensions.values():
    print(d2)

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 36

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_latitude', size = 548

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_longitude', size = 421

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_0', size = 3

<class 'netCDF4._netCDF4.Dimension'>: name = 'pressure', size = 3

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_1', size = 3

<class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2

<class 'netCDF4._netCDF4.Dimension'>: name = 'pressure_0', size = 2

<class 'netCDF4._netCDF4.Dimension'>: name = 'pressure_1', size = 3

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'dim0', size = 3

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_longitude_0', size = 421

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_latitude_0', size = 547

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_3', size = 3



Each variable has a **`dimensions`** and a **`shape`** attribute.

In [132]:
air_temperature.dimensions

('time_0', 'pressure_0', 'grid_latitude', 'grid_longitude')

In [133]:
air_temperature.shape

(3, 2, 548, 421)

### Each dimension typically has a variable associated with it (called a *coordinate* variable).
- *Coordinate variables* are 1D variables that have the same name as dimensions.
- Coordinate variables and *auxiliary coordinate variables* (named by the *coordinates* attribute) locate values in time and space.
- in other words, there is no distinction betweeen 1D variables and dimensions
- in the following example, time, long, lat are all 1D variables i.e. dimensions but the previous variable in the preceding cell called temp was a **`3D variable (or a cube)`**

In [140]:
time = f.variables['time_0'] # a 1D variable
print(time.dimensions) # a ID variable still has a single dimension
print(time.shape) # a 1D variable of course has values
long = f.variables['grid_longitude']  # a 1D variable
print(long.dimensions)
print(long.shape)
lat = f.variables['grid_latitude']  # a 1D variable
print(lat.dimensions)
print(lat.shape)
pressure = f.variables['pressure_0']

('time_0',)
(3,)
('grid_longitude',)
(421,)
('grid_latitude',)
(548,)


## Accessing data from a netCDF variable object

- netCDF variables objects behave much like numpy arrays.
- slicing a netCDF variable object returns a numpy array with the data.
- Boolean array and integer sequence indexing behaves differently for netCDF variables than for numpy arrays. Only 1-d boolean arrays and integer sequences are allowed, and these indices work independently along each dimension (similar to the way vector subscripts work in fortran).

In [135]:
time_value = time[:]  # Reads the netCDF variable MT, array of one element
print(time_value) 
print(time_value.shape)

[390448. 390449. 390450.]
(3,)


In [136]:
long_value = long[:] # examine first 10 values of longitude
print(long_value) 

[354.91074 354.93073 354.95074 354.97073 354.99072 355.01074 355.03073
 355.05075 355.07074 355.09073 355.11075 355.13074 355.15073 355.17075
 355.19073 355.21072 355.23074 355.25073 355.27072 355.29074 355.31073
 355.33075 355.35074 355.37073 355.39075 355.41074 355.43073 355.45074
 355.47073 355.49072 355.51074 355.53073 355.55075 355.57074 355.59073
 355.61075 355.63074 355.65073 355.67075 355.69073 355.71072 355.73074
 355.75073 355.77072 355.79074 355.81073 355.83075 355.85074 355.87073
 355.89075 355.91074 355.93073 355.95074 355.97073 355.99072 356.01074
 356.03073 356.05075 356.07074 356.09073 356.11075 356.13074 356.15073
 356.17075 356.19073 356.21072 356.23074 356.25073 356.27072 356.29074
 356.31073 356.33075 356.35074 356.37073 356.39075 356.41074 356.43073
 356.45074 356.47073 356.49072 356.51074 356.53073 356.55075 356.57074
 356.59073 356.61075 356.63074 356.65073 356.67075 356.69073 356.71072
 356.73074 356.75073 356.77072 356.79074 356.81073 356.83075 356.85074
 356.8

In [139]:
lat_value = lat[:] # examine first 10 values of longitude
print(lat_value) 

[-3.77994990e+00 -3.75994992e+00 -3.73994994e+00 -3.71994996e+00
 -3.69994998e+00 -3.67995000e+00 -3.65995002e+00 -3.63994980e+00
 -3.61994982e+00 -3.59994984e+00 -3.57994986e+00 -3.55994987e+00
 -3.53994989e+00 -3.51994991e+00 -3.49994993e+00 -3.47994995e+00
 -3.45994997e+00 -3.43994999e+00 -3.41995001e+00 -3.39995003e+00
 -3.37995005e+00 -3.35994983e+00 -3.33994985e+00 -3.31994987e+00
 -3.29994988e+00 -3.27994990e+00 -3.25994992e+00 -3.23994994e+00
 -3.21994996e+00 -3.19994998e+00 -3.17995000e+00 -3.15994978e+00
 -3.13994980e+00 -3.11994982e+00 -3.09994984e+00 -3.07994986e+00
 -3.05994987e+00 -3.03994989e+00 -3.01994991e+00 -2.99994993e+00
 -2.97994995e+00 -2.95994997e+00 -2.93994999e+00 -2.91995001e+00
 -2.89995003e+00 -2.87995005e+00 -2.85995007e+00 -2.83994985e+00
 -2.81994987e+00 -2.79994988e+00 -2.77994990e+00 -2.75994992e+00
 -2.73994994e+00 -2.71994996e+00 -2.69994998e+00 -2.67994976e+00
 -2.65994978e+00 -2.63994980e+00 -2.61994982e+00 -2.59994984e+00
 -2.57994986e+00 -2.55994

In [142]:
pressure_value = pressure[:]
print(pressure_value)

[500. 850.]


In [224]:
air_temperature_value = air_temperature[:]
print(air_temperature_value[:])

[[[[257.    256.875 256.875 ... 262.25  262.25  262.25 ]
   [257.    257.    257.    ... 262.25  262.25  262.25 ]
   [257.    257.    257.    ... 262.25  262.25  262.25 ]
   ...
   [259.625 259.625 259.625 ... 255.375 255.375 255.375]
   [259.625 259.625 259.625 ... 255.375 255.375 255.375]
   [259.625 259.625 259.625 ... 255.375 255.375 255.375]]

  [[284.5   284.5   284.625 ... 289.125 289.125 289.125]
   [284.5   284.5   284.625 ... 289.125 289.125 289.125]
   [284.5   284.5   284.625 ... 289.125 289.125 289.   ]
   ...
   [278.75  278.75  278.75  ... 278.125 278.    277.75 ]
   [278.75  278.625 278.625 ... 278.25  278.    277.75 ]
   [278.625 278.625 278.5   ... 278.25  278.    277.875]]]


 [[[257.375 257.375 257.375 ... 262.25  262.25  262.25 ]
   [257.375 257.375 257.375 ... 262.25  262.25  262.25 ]
   [257.375 257.375 257.25  ... 262.25  262.25  262.25 ]
   ...
   [259.875 259.875 259.875 ... 255.75  255.75  255.75 ]
   [259.875 259.875 259.875 ... 255.75  255.75  255.75 ]
   [

## Now, we need to traverse the air_temperature cube

### But first let's see if we can filter out variables with more than one dims (in other words), this is the list of cubes

In [147]:
for key in f.variables.keys():
    num_of_axis = len(f.variables[key].dimensions)
    if num_of_axis > 1:
        print(key)

stratiform_snowfall_rate
cloud_base_altitude_assuming_only_consider_cloud_area_fraction_greater_than_2p5_oktas
cloud_area_fraction_assuming_maximum_random_overlap
wet_bulb_freezing_level_altitude
wet_bulb_potential_temperature
unknown
air_pressure_at_sea_level
air_temperature
air_temperature_0
time_1_bnds
forecast_period_1_bnds
air_temperature_1
air_temperature_2
dew_point_temperature
fog_area_fraction
geopotential_height
high_type_cloud_area_fraction
low_type_cloud_area_fraction
medium_type_cloud_area_fraction
relative_humidity
relative_humidity_0
specific_humidity
stratiform_rainfall_amount
forecast_period_2_bnds
time_2_bnds
stratiform_rainfall_rate
stratiform_snowfall_amount
surface_air_pressure
surface_downward_eastward_stress
surface_downward_northward_stress
surface_downwelling_longwave_flux
surface_downwelling_shortwave_flux_in_air
visibility_in_air
wind_speed_of_gust
x_wind
x_wind_0
x_wind_1
y_wind
y_wind_0
y_wind_1


In [149]:
for key in f.variables.keys():
    if key.startswith('air_temperature'):
        print(key)

air_temperature
air_temperature_0
air_temperature_1
air_temperature_2


In [154]:
print(f.variables['air_temperature'].shape)

(3, 548, 421)


In [155]:
print(f.variables['air_temperature_0'].shape)

(3, 548, 421)


In [156]:
print(f.variables['air_temperature_1'].shape)

(3, 548, 421)


In [ ]:
print(f.variables['air_temperature_2'].shape)

**This proves that the arrays are not similar**

In [160]:
f.variables['air_temperature'] == f.variables['air_temperature_1']

False

In [162]:
print(f.variables['air_temperature'][0:10])

[[[289.5   289.5   289.5   ... 304.125 304.25  304.125]
  [289.5   289.5   289.5   ... 304.125 304.    303.625]
  [289.5   289.5   289.5   ... 303.875 303.75  303.25 ]
  ...
  [287.25  287.25  287.25  ... 289.125 289.125 289.125]
  [287.25  287.25  287.25  ... 289.    289.    289.125]
  [287.25  287.25  287.25  ... 289.    289.125 289.125]]

 [[289.5   289.5   289.5   ... 303.75  303.875 303.75 ]
  [289.5   289.5   289.5   ... 303.75  303.75  303.25 ]
  [289.5   289.5   289.5   ... 303.625 303.375 302.875]
  ...
  [287.375 287.375 287.375 ... 289.125 289.125 289.25 ]
  [287.375 287.375 287.375 ... 289.125 289.25  289.25 ]
  [287.25  287.25  287.375 ... 289.25  289.25  289.125]]

 [[289.375 289.375 289.375 ... 302.375 302.625 302.625]
  [289.375 289.375 289.375 ... 302.625 302.5   301.875]
  [289.375 289.375 289.375 ... 302.625 302.25  301.625]
  ...
  [287.375 287.375 287.375 ... 289.    289.125 289.125]
  [287.375 287.375 287.375 ... 289.125 289.125 289.125]
  [287.375 287.375 287.375

In [163]:
print(f.variables['air_temperature_0'][0:10])

[[[289.5   289.5   289.5   ... 304.125 304.25  304.125]
  [289.5   289.5   289.5   ... 304.125 304.    303.625]
  [289.5   289.5   289.5   ... 304.    303.75  303.375]
  ...
  [287.25  287.375 287.375 ... 289.125 289.125 289.125]
  [287.375 287.375 287.375 ... 289.125 289.125 289.125]
  [287.375 287.375 287.375 ... 289.125 289.125 289.25 ]]

 [[289.5   289.5   289.5   ... 304.125 304.25  304.125]
  [289.5   289.5   289.5   ... 304.125 304.    303.625]
  [289.5   289.5   289.5   ... 303.875 303.75  303.25 ]
  ...
  [287.375 287.375 287.375 ... 289.125 289.125 289.25 ]
  [287.375 287.375 287.375 ... 289.25  289.25  289.25 ]
  [287.25  287.25  287.375 ... 289.25  289.25  289.125]]

 [[289.5   289.5   289.5   ... 303.75  303.875 303.75 ]
  [289.5   289.5   289.5   ... 303.75  303.625 303.25 ]
  [289.5   289.5   289.5   ... 303.625 303.375 302.875]
  ...
  [287.375 287.375 287.375 ... 289.125 289.25  289.25 ]
  [287.375 287.375 287.375 ... 289.125 289.25  289.25 ]
  [287.375 287.375 287.375

### Now that we know they are different, we need to find out why. Since the data is different, may be it is for a different date/time period and for different long and lat values. Let's find out.

In [171]:
temp = f.variables['air_temperature']
temp0 = f.variables['air_temperature_0']
temp1 = f.variables['air_temperature_1']
temp2 = f.variables['air_temperature_2']
print("temp: " + str(temp.dimensions))
print(temp.shape)
print("===================")
print("temp0: " + str(temp0.dimensions))
print(temp0.shape)
print("===================")
print("temp1: " + str(temp1.dimensions))
print(temp1.shape)
print("===================")
print("temp2: " + str(temp2.dimensions))
print(temp2.shape)

temp: ('time_0', 'grid_latitude', 'grid_longitude')
(3, 548, 421)
temp0: ('time_1', 'grid_latitude', 'grid_longitude')
(3, 548, 421)
temp1: ('time_1', 'grid_latitude', 'grid_longitude')
(3, 548, 421)
temp2: ('time_0', 'pressure_0', 'grid_latitude', 'grid_longitude')
(3, 2, 548, 421)


In [172]:
temp0 == temp1

False

In [177]:
print(temp0[:][0:1:1])

[[[289.5   289.5   289.5   ... 304.125 304.25  304.125]
  [289.5   289.5   289.5   ... 304.125 304.    303.625]
  [289.5   289.5   289.5   ... 304.    303.75  303.375]
  ...
  [287.25  287.375 287.375 ... 289.125 289.125 289.125]
  [287.375 287.375 287.375 ... 289.125 289.125 289.125]
  [287.375 287.375 287.375 ... 289.125 289.125 289.25 ]]]


In [178]:
print(temp1[:][0:1:1])

[[[289.5   289.5   289.5   ... 304.125 304.125 304.   ]
  [289.5   289.5   289.5   ... 304.125 304.    303.5  ]
  [289.5   289.5   289.5   ... 303.875 303.75  303.25 ]
  ...
  [287.25  287.25  287.25  ... 289.    289.    289.   ]
  [287.25  287.25  287.25  ... 289.    289.    289.   ]
  [287.25  287.25  287.25  ... 289.    289.    289.   ]]]


**Right off the bat, it seems like there are different time dimensions attached to temp0 than temp. Let's see what the difference is between those two**

In [210]:
time_0 = f.variables['time_0']
time_1 = f.variables['time_1']
print(time_0[:])
print(time_1[:])

[390448. 390449. 390450.]
[390447.5 390448.5 390449.5]


#### What's clear from this is temp, temp0 and temp2 are all different, however temp0 and temp1 look very similar i.e. they have the exact same dimensions, but the data itself is different. I am going to assume that one is the update of next one

In [182]:
print(temp)
print("---------------------------------------------------------")
print(temp0)
print("---------------------------------------------------------")
print(temp1)
print("---------------------------------------------------------")
print(temp2)

<class 'netCDF4._netCDF4.Variable'>
float32 air_temperature(time_0, grid_latitude, grid_longitude)
    _FillValue: -1073741800.0
    standard_name: air_temperature
    units: K
    um_stash_source: m01s03i236
    grid_mapping: rotated_latitude_longitude
    coordinates: forecast_period_0 forecast_reference_time height
unlimited dimensions: time_0
current shape = (3, 548, 421)
filling on
---------------------------------------------------------
<class 'netCDF4._netCDF4.Variable'>
float32 air_temperature_0(time_1, grid_latitude, grid_longitude)
    _FillValue: -1073741800.0
    standard_name: air_temperature
    units: K
    um_stash_source: m01s03i236
    cell_methods: time_1: maximum (interval: 1 hour)
    grid_mapping: rotated_latitude_longitude
    coordinates: forecast_period_1 forecast_reference_time height
unlimited dimensions: time_1
current shape = (3, 548, 421)
filling on
---------------------------------------------------------
<class 'netCDF4._netCDF4.Variable'>
float32 air_t

## AHA !!!! Look closely and you will see that difference is in this thing called `cell methods`. On temp0, it is maximum and on temp1, it is minium. So temp0 is the max temp for that time frame and temp1, it is the min temp for the same frame.


### Alright, enough dkcng around for now.  Let's get this sucker in tabular format finally. Focussing on just a single one at this point. There are a total of 972 values across the three dimensions in air_temperature i.e. 

`3 + 548 + 421 = 972`

In [189]:
for dim in temp.dimensions:
    for data in f.variables[dim][:]:
        print(data)

390448.0
390449.0
390450.0
-3.77995
-3.75995
-3.73995
-3.71995
-3.69995
-3.67995
-3.65995
-3.6399498
-3.6199498
-3.5999498
-3.5799499
-3.5599499
-3.53995
-3.51995
-3.49995
-3.47995
-3.45995
-3.43995
-3.41995
-3.39995
-3.37995
-3.3599498
-3.3399498
-3.3199499
-3.29995
-3.27995
-3.25995
-3.23995
-3.21995
-3.19995
-3.17995
-3.1599498
-3.1399498
-3.1199498
-3.0999498
-3.0799499
-3.0599499
-3.03995
-3.01995
-2.99995
-2.97995
-2.95995
-2.93995
-2.91995
-2.89995
-2.87995
-2.85995
-2.8399498
-2.8199499
-2.79995
-2.77995
-2.75995
-2.73995
-2.71995
-2.69995
-2.6799498
-2.6599498
-2.6399498
-2.6199498
-2.5999498
-2.5799499
-2.5599499
-2.53995
-2.51995
-2.49995
-2.47995
-2.45995
-2.43995
-2.41995
-2.39995
-2.37995
-2.35995
-2.33995
-2.31995
-2.29995
-2.27995
-2.25995
-2.23995
-2.21995
-2.19995
-2.17995
-2.1599498
-2.1399498
-2.1199498
-2.0999498
-2.0799499
-2.0599499
-2.03995
-2.01995
-1.9999499
-1.97995
-1.95995
-1.93995
-1.9199499
-1.8999499
-1.8799499
-1.85995
-1.83995
-1.81995
-1.79995
-1.7799

**These 972 values make up a total of 692124 total distinct temperature readings which is obvious from the total size of the cube itself i.e. 

`3 X 548 X 421 = 692124`

In [188]:
print(temp[:].size)

692124


### So i need to basically run a for loop 692124 times to grab all the values

In [236]:
air_temperature = f.variables['air_temperature'][:]
l1,l2,l3 = air_temperature.shape
for i in range(l1-1):
    for j in range(l2-1):
        for k in range(l3-1):
            print(i,j,k)
            print(air_temperature[i,j,k])
        
        
        
    

0 0 0
289.5
0 0 1
289.5
0 0 2
289.5
0 0 3
289.5
0 0 4
289.5
0 0 5
289.5
0 0 6
289.5
0 0 7
289.5
0 0 8
289.5
0 0 9
289.5
0 0 10
289.5
0 0 11
289.625
0 0 12
289.625
0 0 13
289.625
0 0 14
289.625
0 0 15
289.625
0 0 16
289.625
0 0 17
289.625
0 0 18
289.625
0 0 19
289.625
0 0 20
289.625
0 0 21
289.625
0 0 22
289.625
0 0 23
289.625
0 0 24
289.625
0 0 25
289.75
0 0 26
289.75
0 0 27
289.75
0 0 28
289.75
0 0 29
289.75
0 0 30
289.75
0 0 31
289.75
0 0 32
289.75
0 0 33
289.75
0 0 34
289.75
0 0 35
289.75
0 0 36
289.875
0 0 37
289.875
0 0 38
289.875
0 0 39
289.875
0 0 40
289.875
0 0 41
289.875
0 0 42
289.875
0 0 43
289.875
0 0 44
289.875
0 0 45
290.0
0 0 46
290.0
0 0 47
290.0
0 0 48
290.0
0 0 49
290.0
0 0 50
290.0
0 0 51
290.0
0 0 52
290.0
0 0 53
290.0
0 0 54
290.0
0 0 55
290.125
0 0 56
290.125
0 0 57
290.125
0 0 58
290.125
0 0 59
290.125
0 0 60
290.125
0 0 61
290.125
0 0 62
290.125
0 0 63
290.125
0 0 64
290.25
0 0 65
290.25
0 0 66
290.25
0 0 67
290.25
0 0 68
290.25
0 0 69
290.25
0 0 70
290.25
0 0 7

0 1 198
290.875
0 1 199
290.875
0 1 200
290.875
0 1 201
291.0
0 1 202
291.0
0 1 203
291.125
0 1 204
291.375
0 1 205
291.375
0 1 206
291.375
0 1 207
291.375
0 1 208
291.5
0 1 209
291.625
0 1 210
291.625
0 1 211
291.75
0 1 212
291.75
0 1 213
291.875
0 1 214
292.125
0 1 215
292.375
0 1 216
292.625
0 1 217
293.125
0 1 218
294.0
0 1 219
295.0
0 1 220
297.625
0 1 221
297.25
0 1 222
297.25
0 1 223
297.375
0 1 224
297.25
0 1 225
297.0
0 1 226
297.0
0 1 227
297.125
0 1 228
297.25
0 1 229
297.125
0 1 230
296.875
0 1 231
296.625
0 1 232
296.25
0 1 233
296.125
0 1 234
296.0
0 1 235
295.75
0 1 236
295.75
0 1 237
295.625
0 1 238
295.375
0 1 239
294.75
0 1 240
293.25
0 1 241
293.5
0 1 242
293.5
0 1 243
293.625
0 1 244
293.625
0 1 245
293.625
0 1 246
293.625
0 1 247
293.625
0 1 248
293.75
0 1 249
293.75
0 1 250
293.75
0 1 251
293.75
0 1 252
293.875
0 1 253
293.875
0 1 254
293.875
0 1 255
294.0
0 1 256
294.0
0 1 257
294.125
0 1 258
294.125
0 1 259
294.125
0 1 260
294.125
0 1 261
294.125
0 1 262
294.25


303.25
0 2 411
303.375
0 2 412
303.625
0 2 413
304.0
0 2 414
304.5
0 2 415
304.75
0 2 416
304.375
0 2 417
304.0
0 2 418
303.875
0 2 419
303.75
0 3 0
289.5
0 3 1
289.5
0 3 2
289.5
0 3 3
289.5
0 3 4
289.5
0 3 5
289.5
0 3 6
289.5
0 3 7
289.5
0 3 8
289.5
0 3 9
289.5
0 3 10
289.5
0 3 11
289.625
0 3 12
289.625
0 3 13
289.625
0 3 14
289.625
0 3 15
289.625
0 3 16
289.625
0 3 17
289.625
0 3 18
289.625
0 3 19
289.625
0 3 20
289.625
0 3 21
289.625
0 3 22
289.625
0 3 23
289.625
0 3 24
289.75
0 3 25
289.75
0 3 26
289.75
0 3 27
289.75
0 3 28
289.75
0 3 29
289.75
0 3 30
289.75
0 3 31
289.75
0 3 32
289.75
0 3 33
289.75
0 3 34
289.875
0 3 35
289.875
0 3 36
289.875
0 3 37
289.875
0 3 38
289.875
0 3 39
289.875
0 3 40
289.875
0 3 41
289.875
0 3 42
289.875
0 3 43
289.875
0 3 44
290.0
0 3 45
290.0
0 3 46
290.0
0 3 47
290.0
0 3 48
290.0
0 3 49
290.0
0 3 50
290.0
0 3 51
290.0
0 3 52
290.0
0 3 53
290.0
0 3 54
290.125
0 3 55
290.125
0 3 56
290.125
0 3 57
290.125
0 3 58
290.125
0 3 59
290.125
0 3 60
290.125
0 3 

0 4 237
295.0
0 4 238
293.25
0 4 239
293.5
0 4 240
293.5
0 4 241
293.5
0 4 242
293.5
0 4 243
293.5
0 4 244
293.5
0 4 245
293.5
0 4 246
293.625
0 4 247
293.625
0 4 248
293.625
0 4 249
293.625
0 4 250
293.75
0 4 251
293.75
0 4 252
293.75
0 4 253
293.875
0 4 254
293.875
0 4 255
293.875
0 4 256
294.0
0 4 257
294.0
0 4 258
294.0
0 4 259
294.125
0 4 260
294.25
0 4 261
294.25
0 4 262
294.25
0 4 263
294.375
0 4 264
294.375
0 4 265
294.375
0 4 266
294.5
0 4 267
294.625
0 4 268
294.625
0 4 269
294.625
0 4 270
294.5
0 4 271
294.5
0 4 272
294.5
0 4 273
294.75
0 4 274
295.125
0 4 275
295.75
0 4 276
296.25
0 4 277
296.5
0 4 278
296.875
0 4 279
297.125
0 4 280
297.375
0 4 281
297.625
0 4 282
298.0
0 4 283
298.625
0 4 284
299.5
0 4 285
300.25
0 4 286
305.0
0 4 287
304.75
0 4 288
304.5
0 4 289
304.25
0 4 290
304.125
0 4 291
304.0
0 4 292
304.0
0 4 293
304.0
0 4 294
304.0
0 4 295
303.75
0 4 296
303.625
0 4 297
303.375
0 4 298
303.375
0 4 299
303.25
0 4 300
303.125
0 4 301
303.0
0 4 302
302.75
0 4 303
30

0 6 73
290.375
0 6 74
290.375
0 6 75
290.375
0 6 76
290.375
0 6 77
290.375
0 6 78
290.375
0 6 79
290.375
0 6 80
290.375
0 6 81
290.5
0 6 82
290.5
0 6 83
290.5
0 6 84
290.5
0 6 85
290.5
0 6 86
290.5
0 6 87
290.5
0 6 88
290.5
0 6 89
290.625
0 6 90
290.625
0 6 91
290.625
0 6 92
290.625
0 6 93
290.625
0 6 94
290.625
0 6 95
290.625
0 6 96
290.625
0 6 97
290.625
0 6 98
290.625
0 6 99
290.625
0 6 100
290.625
0 6 101
290.625
0 6 102
290.625
0 6 103
290.625
0 6 104
290.625
0 6 105
290.625
0 6 106
290.625
0 6 107
290.625
0 6 108
290.625
0 6 109
290.625
0 6 110
290.625
0 6 111
290.625
0 6 112
290.625
0 6 113
290.625
0 6 114
290.75
0 6 115
290.75
0 6 116
290.75
0 6 117
290.75
0 6 118
290.75
0 6 119
290.75
0 6 120
290.625
0 6 121
290.625
0 6 122
290.625
0 6 123
290.625
0 6 124
290.625
0 6 125
290.625
0 6 126
290.625
0 6 127
290.625
0 6 128
290.625
0 6 129
290.625
0 6 130
290.625
0 6 131
290.625
0 6 132
290.625
0 6 133
290.625
0 6 134
290.625
0 6 135
290.625
0 6 136
290.625
0 6 137
290.625
0 6 138
2

...

0 16 63
290.125
0 16 64
290.125
0 16 65
290.125
0 16 66
290.125
0 16 67
290.25
0 16 68
290.25
0 16 69
290.25
0 16 70
290.25
0 16 71
290.25
0 16 72
290.25
0 16 73
290.25
0 16 74
290.25
0 16 75
290.25
0 16 76
290.25
0 16 77
290.375
0 16 78
290.375
0 16 79
290.375
0 16 80
290.375
0 16 81
290.375
0 16 82
290.375
0 16 83
290.375
0 16 84
290.375
0 16 85
290.375
0 16 86
290.375
0 16 87
290.375
0 16 88
290.375
0 16 89
290.5
0 16 90
290.5
0 16 91
290.5
0 16 92
290.5
0 16 93
290.5
0 16 94
290.5
0 16 95
290.5
0 16 96
290.5
0 16 97
290.5
0 16 98
290.5
0 16 99
290.5
0 16 100
290.5
0 16 101
290.5
0 16 102
290.5
0 16 103
290.5
0 16 104
290.5
0 16 105
290.5
0 16 106
290.5
0 16 107
290.5
0 16 108
290.5
0 16 109
290.5
0 16 110
290.5
0 16 111
290.5
0 16 112
290.5
0 16 113
290.5
0 16 114
290.5
0 16 115
290.5
0 16 116
290.5
0 16 117
290.5
0 16 118
290.5
0 16 119
290.5
0 16 120
290.5
0 16 121
290.5
0 16 122
290.5
0 16 123
290.5
0 16 124
290.5
0 16 125
290.5
0 16 126
290.5
0 16 127
290.5
0 16 128
290.5
0 16 

0 17 351
303.5
0 17 352
303.25
0 17 353
303.25
0 17 354
303.375
0 17 355
303.5
0 17 356
303.5
0 17 357
303.375
0 17 358
303.375
0 17 359
303.5
0 17 360
303.5
0 17 361
303.5
0 17 362
303.5
0 17 363
303.375
0 17 364
303.375
0 17 365
303.375
0 17 366
303.25
0 17 367
303.375
0 17 368
303.375
0 17 369
303.375
0 17 370
303.375
0 17 371
303.375
0 17 372
303.375
0 17 373
303.375
0 17 374
303.375
0 17 375
303.25
0 17 376
303.25
0 17 377
303.25
0 17 378
303.375
0 17 379
303.375
0 17 380
303.375
0 17 381
303.625
0 17 382
303.875
0 17 383
303.875
0 17 384
303.75
0 17 385
303.375
0 17 386
303.375
0 17 387
303.375
0 17 388
303.375
0 17 389
303.625
0 17 390
303.75
0 17 391
304.0
0 17 392
304.25
0 17 393
304.125
0 17 394
303.875
0 17 395
304.25
0 17 396
304.375
0 17 397
304.375
0 17 398
304.375
0 17 399
304.5
0 17 400
304.625
0 17 401
304.5
0 17 402
304.0
0 17 403
304.125
0 17 404
304.75
0 17 405
305.125
0 17 406
305.125
0 17 407
305.125
0 17 408
305.5
0 17 409
305.5
0 17 410
305.625
0 17 411
306.0
0 

294.5
0 19 275
294.625
0 19 276
294.625
0 19 277
294.75
0 19 278
294.875
0 19 279
295.0
0 19 280
295.25
0 19 281
295.625
0 19 282
296.375
0 19 283
297.375
0 19 284
300.125
0 19 285
300.0
0 19 286
299.625
0 19 287
299.375
0 19 288
299.125
0 19 289
299.125
0 19 290
299.0
0 19 291
298.75
0 19 292
298.75
0 19 293
298.875
0 19 294
299.25
0 19 295
299.5
0 19 296
299.625
0 19 297
299.75
0 19 298
300.0
0 19 299
300.375
0 19 300
300.625
0 19 301
300.625
0 19 302
300.5
0 19 303
300.125
0 19 304
299.75
0 19 305
299.5
0 19 306
299.5
0 19 307
299.875
0 19 308
300.0
0 19 309
299.875
0 19 310
299.5
0 19 311
299.375
0 19 312
299.5
0 19 313
300.0
0 19 314
300.5
0 19 315
301.5
0 19 316
302.5
0 19 317
303.375
0 19 318
303.875
0 19 319
304.0
0 19 320
304.125
0 19 321
304.0
0 19 322
304.0
0 19 323
304.25
0 19 324
304.375
0 19 325
304.375
0 19 326
304.125
0 19 327
304.0
0 19 328
304.125
0 19 329
304.375
0 19 330
304.5
0 19 331
304.625
0 19 332
304.5
0 19 333
304.625
0 19 334
304.625
0 19 335
304.75
0 19 336

0 21 134
290.5
0 21 135
290.5
0 21 136
290.5
0 21 137
290.375
0 21 138
290.375
0 21 139
290.375
0 21 140
290.375
0 21 141
290.375
0 21 142
290.375
0 21 143
290.375
0 21 144
290.375
0 21 145
290.375
0 21 146
290.375
0 21 147
290.375
0 21 148
290.375
0 21 149
290.375
0 21 150
290.375
0 21 151
290.375
0 21 152
290.25
0 21 153
290.25
0 21 154
290.25
0 21 155
290.25
0 21 156
290.25
0 21 157
290.25
0 21 158
290.25
0 21 159
290.25
0 21 160
290.25
0 21 161
290.25
0 21 162
290.25
0 21 163
290.25
0 21 164
290.25
0 21 165
290.25
0 21 166
290.25
0 21 167
290.25
0 21 168
290.25
0 21 169
290.25
0 21 170
290.25
0 21 171
290.25
0 21 172
290.25
0 21 173
290.25
0 21 174
290.25
0 21 175
290.25
0 21 176
290.25
0 21 177
290.25
0 21 178
290.25
0 21 179
290.25
0 21 180
290.375
0 21 181
290.375
0 21 182
290.375
0 21 183
290.375
0 21 184
290.375
0 21 185
290.375
0 21 186
290.375
0 21 187
290.375
0 21 188
290.375
0 21 189
290.5
0 21 190
290.5
0 21 191
290.5
0 21 192
290.5
0 21 193
290.5
0 21 194
290.5
0 21 195


KeyboardInterrupt: 

## or better yet, just reshape the whole array into a single column

In [244]:
reshaped_air_temp = air_temperature.reshape(692124,1)
print(reshaped_air_temp.shape)

(692124, 1)
